
# ✅ Tests API — *AIC111* (projet **test-inference**)

Ce notebook sert de **pense-bête** pour **copier/coller** les commandes de test de ton API, **dans le cloud** (API Gateway) et **en local** (SAM + Docker).  
> ⚠️ Les cellules ci‑dessous sont surtout du **texte et des commandes Bash** à exécuter dans **Git Bash** (ou un terminal). Le notebook n’est pas destiné à être exécuté tel quel dans Jupyter.

---



## 📁 Emplacement du projet SAM

Assure‑toi d’être dans le dossier du projet **SAM** avant d’exécuter les commandes :

```bash
cd "C:/Users/jp202/Documents/Coding Github/Codeboxx/Module11_AIC111/Codeboxx_AIC111/test-inference"
```



## ☁️ Tests **API déployée** (Cloud – API Gateway)

### A) Test rapide (URL actuelle)
```bash
# 1) URL actuelle de l’API (Prod)
API_URL="https://seyq4lqbog.execute-api.us-east-1.amazonaws.com/Prod/classify_digit/"

# 2) Appel HTTP (attendu: 200 + {"prediction": 7.0})
curl -i -X POST "$API_URL"   -H 'Content-Type: application/json'   -d '{"x": 2}'

# Tests supplémentaires
curl -s -X POST "$API_URL" -H 'Content-Type: application/json' -d '{"x": 0}'   # -> {"prediction": 1.0}
curl -s -X POST "$API_URL" -H 'Content-Type: application/json' -d '{"x": 10}'  # -> {"prediction": 31.0}
```



### B) (Optionnel) Récupérer l’URL via CloudFormation puis tester
> Pratique si l’URL change après un nouveau déploiement.
```bash
# Utiliser le profil AWS configuré
export AWS_PROFILE=sam-deployer

# Récupérer l’URL de sortie du stack
API_URL=$(aws cloudformation describe-stacks --stack-name test-inference   --query "Stacks[0].Outputs[?OutputKey=='InferenceApi'].OutputValue" --output text)
echo "$API_URL"

# Tester
curl -i -X POST "$API_URL"   -H 'Content-Type: application/json'   -d '{"x": 2}'
```



### C) (Optionnel) Voir les logs **Lambda** en direct
```bash
sam logs -n InferenceFunction --stack-name test-inference --tail
```



---

## 🐳 Tests **en local** (SAM + Docker)

> Nécessite **Docker Desktop** démarré.

```bash
# Se placer dans le projet
cd "C:/Users/jp202/Documents/Coding Github/Codeboxx/Module11_AIC111/Codeboxx_AIC111/test-inference"

# Rebuild de l’image Lambda (CPU-only)
sam build --use-container

# Lancer l’API locale (laisse cette fenêtre ouverte)
sam local start-api
```
Dans **une autre** fenêtre Git Bash :
```bash
# Appel local (attendu: {"prediction": 7.0})
curl -i -X POST http://127.0.0.1:3000/classify_digit   -H 'Content-Type: application/json'   -d '{"x": 2}'
```



### (Optionnel) Invocation directe via un *event* JSON
```bash
# Créer un event compatible API Gateway (v1)
cat > events/ping.json <<'JSON'
{
  "resource": "/classify_digit",
  "path": "/classify_digit",
  "httpMethod": "POST",
  "headers": { "Content-Type": "application/json" },
  "isBase64Encoded": false,
  "body": "{"x": 2}"
}
JSON

# Invoquer la Lambda localement
sam local invoke InferenceFunction -e events/ping.json
# ...la sortie doit contenir: "body": "{"prediction": 7.0}"
```



---

## ℹ️ Détails utiles

- **Stack CloudFormation** : `test-inference`  
- **Fonction Lambda** : `InferenceFunction`  
- **Profil AWS** : `sam-deployer` (déjà configuré sur ta machine)  
- **Endpoint (Prod)** : `https://seyq4lqbog.execute-api.us-east-1.amazonaws.com/Prod/classify_digit/`  
- **Payload attendu** : `{"x": <nombre>}` → Réponse : `{"prediction": 3*x + 1}`  
- **Chemin projet** : `.../Codeboxx_AIC111/test-inference`  
- **Fichiers clés** : `template.yaml`, `app/app.py`, `app/requirements.txt`, `app/Dockerfile`  
